# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [71]:
# import libraries

# General libraries
import pandas as pd
import numpy as np
import re

# Database libraries
from sqlalchemy import create_engine

# Tokenization libraries
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger', 'stopwords'])

# ML libraries
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.multioutput import MultiOutputClassifier, MultiOutputRegressor
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split
from sklearn.compose import make_column_transformer
from sklearn.metrics import confusion_matrix
from sklearn.metrics import multilabel_confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/gabrielgarciaramirez/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/gabrielgarciaramirez/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/gabrielgarciaramirez/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/gabrielgarciaramirez/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [59]:
# load data from database
engine = create_engine('sqlite:///disaster_pipeline.db')
df = pd.read_sql('messages_categories', engine)
X = df['message']
y = df.iloc[:,4:]

### 2. Write a tokenization function to process your text data

In [60]:
def tokenize(text):
    
    lemmatizer = WordNetLemmatizer()
    
    # Stop words definition
    stop_words = stopwords.words("english")
    
    # Normalization case and punctuation removal
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    
    # Tokenization of words
    tokens = word_tokenize(text)
    
    # Stop words removal
    tokens = [lemmatizer.lemmatize(w) for w in tokens if w not in stop_words]
    
    return tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [61]:
pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(LogisticRegression(max_iter=2000))),
    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

#### Recreation of the pipeline step by step

This was done to be able to understand how the pipeline was going to be created

In [142]:
# WITHOUT PIPELINE
vect = CountVectorizer(tokenizer=tokenize)
tfidf = TfidfTransformer()
clf = MultiOutputClassifier(LogisticRegression(max_iter=2000))

# First, it was tried to use the genre column also, but it got messy when trying to get
    # parameters to measure the model 
    
# transformer = make_column_transformer((vect, 'message'), (vect, 'genre'))

# train classifier
X_train_counts = transformer.fit_transform(X_train)
# display(X_train_counts.toarray())
X_train_tfidf = tfidf.fit_transform(X_train_counts)
#display(X_train_tfidf.toarray())
clf.fit(X_train_tfidf, y_train)

# predict on test data
X_test_counts = transformer.transform(X_test)
X_test_tfidf = tfidf.transform(X_test_counts)
y_pred = clf.predict(X_test_tfidf)

#### Accuracy results of pipeline 'from a scratch'

In [151]:
# predict on test data
X_test_counts = transformer.transform(X_test)
X_test_tfidf = tfidf.transform(X_test_counts)
y_pred = clf.predict(X_test_tfidf)


accuracy = (y_pred == y_test).mean().mean()

print("Accuracy:", accuracy)

Accuracy: 0.9497003378010243


#### Actual pipeline training

Now, we train the model using the actual pipeline built in step 3

In [62]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8)

# train classifier
pipeline.fit(X_train, y_train)

# predict on test data
predicted = pipeline.predict(X_test)

#### Displaying of vocabularies

As the transformer is a multicolumn type, all the vocabularies are obtained by accesing the array. The genre column was not used since it generated a multiclass-multioutput that messed up getting the performance of the model

In [19]:
# When using columnn transformer as a step

#print("1 \n", sorted(transformer.transformers_[0][1].vocabulary_, key=lambda item: item[0]))
#print("\n 2 \n", sorted(transformer.transformers_[1][1].vocabulary_, key=lambda item: item[1]))

# When we just use CountVectorizer

print(pipeline.named_steps['vect'].vocabulary_)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [69]:
predicted_df = pd.DataFrame(predicted, columns=y_test.columns)

target_names = ['class 0', 'class 1']

for column in y_train.columns:
    report = classification_report(y_test[column], predicted_df[column], target_names=target_names, zero_division=0)
    print("\n Classification report for column '{}': \n {}".format(column, report))


 Classification report for column 'related': 
               precision    recall  f1-score   support

     class 0       0.70      0.40      0.51      1265
     class 1       0.83      0.95      0.88      3941

    accuracy                           0.81      5206
   macro avg       0.77      0.67      0.70      5206
weighted avg       0.80      0.81      0.79      5206


 Classification report for column 'request': 
               precision    recall  f1-score   support

     class 0       0.91      0.98      0.94      4325
     class 1       0.83      0.53      0.64       881

    accuracy                           0.90      5206
   macro avg       0.87      0.75      0.79      5206
weighted avg       0.90      0.90      0.89      5206


 Classification report for column 'offer': 
               precision    recall  f1-score   support

     class 0       1.00      1.00      1.00      5181
     class 1       0.00      0.00      0.00        25

    accuracy                           1

### 6. Improve your model
Use grid search to find better parameters. 

In [72]:
parameters = {
    'vect__ngram_range': ((1,1),(1,2),(2,2)),
    'tfidf__use_idf': [True, False],
    'tfidf__smooth_idf': [True, False],
    'tfidf__sublinear_tf': [True, False]
}

cv = GridSearchCV(pipeline, param_grid=parameters)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [73]:
target_names = ['class 0', 'class 1']

for column in y_train.columns:
    report = classification_report(y_test[column], predicted_df[column], target_names=target_names, zero_division=0)
    print("\n Classification report for column '{}': \n {}".format(column, report))


 Classification report for column 'related': 
               precision    recall  f1-score   support

     class 0       0.70      0.40      0.51      1265
     class 1       0.83      0.95      0.88      3941

    accuracy                           0.81      5206
   macro avg       0.77      0.67      0.70      5206
weighted avg       0.80      0.81      0.79      5206


 Classification report for column 'request': 
               precision    recall  f1-score   support

     class 0       0.91      0.98      0.94      4325
     class 1       0.83      0.53      0.64       881

    accuracy                           0.90      5206
   macro avg       0.87      0.75      0.79      5206
weighted avg       0.90      0.90      0.89      5206


 Classification report for column 'offer': 
               precision    recall  f1-score   support

     class 0       1.00      1.00      1.00      5181
     class 1       0.00      0.00      0.00        25

    accuracy                           1

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.